In [1]:
# Add directory above current directory to path
import sys as SYS; SYS.path.insert(0, '..')

# for saving
import os

from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

from src import setup
# setup.use_gpu()

In [2]:
from src import ( 
    measurements as measure,
    density_matrix as DM,
    simulation as sim,
    orders,
    order_rules,
    random_unitary,
    simulation)

from Scripts import simulation_CLI as cleo

In [3]:
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors 

import matplotlib.animation as animation
# Add directory above current directory to path
import sys as SYS; SYS.path.insert(0, '../..')
import h5py
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from src import ket as ket
from src import density_matrix as DM

In [31]:
Trials = 100
for unitary_seed in range(Trials):
    ##initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]

    initial_pops = [0,0,0,0,1,0,0,0]
    #initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c2_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "random"
    num_steps = 500
    data = cleo.execute(file_name="random",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9);

90%
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
90%
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unit

In [32]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]


    initial_pops = [0,0,0,0,1,0,0,0]
#    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c4_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "random"
    num_steps = 500
    data = cleo.execute(file_name="random",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
90%
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unit

In [33]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    initial_pops = [0,0,0,0,1,0,0,0]
#    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c5_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "random"
    num_steps = 500
    data = cleo.execute(file_name="random",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
90%
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unit

In [4]:
Trials = 5
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    #initial_pops = [0,0,0,0,1,0,0,0]
    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c6_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "random"
    num_steps = 500
    data = cleo.execute(file_name="random",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
90%
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unit

In [4]:
Trials = 5
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]
    #initial_pops = [0,0,0,0,1,0,0,0]
    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c7"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 4
    order_rule_name = "random"
    num_steps = 500
    data = cleo.execute(file_name="random",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="unitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c7_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c7_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c7_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
90%
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c7_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c7_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c7_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_random
90%
simu

In [6]:
for chunk in orders.n_random_c7_orders(8,8, seed=None, chunk_size=4)[1]:
    print(len(chunk))


4
4


In [21]:
import sys as SYS; SYS.path.insert(0, '../..')
from Scripts import hdf5merge
hdf5merge.merge_hdf5_files("../data/random")

In [37]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    initial_pops = [0,0,0,0,1,0,0,0]
#    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c2_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "greedy"
    num_steps = 500
    data = cleo.execute(file_name="greedy",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unit

In [38]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    initial_pops = [0,0,0,0,1,0,0,0]
#    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c4_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "greedy"
    num_steps = 500
    data = cleo.execute(file_name="greedy",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unit

In [39]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    initial_pops = [0,0,0,0,1,0,0,0]
#    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c5_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "greedy"
    num_steps = 500
    data = cleo.execute(file_name="greedy",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unit

In [40]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    initial_pops = [0,0,0,0,1,0,0,0]
#    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c6_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "greedy"
    num_steps = 500
    data = cleo.execute(file_name="greedy",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unit

In [41]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    initial_pops = [0,0,0,0,1,0,0,0]
#    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "cN_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "greedy"
    num_steps = 500
    data = cleo.execute(file_name="greedy",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unit

In [42]:
import sys as SYS; SYS.path.insert(0, '../..')
from Scripts import hdf5merge
hdf5merge.merge_hdf5_files("../data/greedy")

In [43]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    initial_pops = [0,0,0,0,1,0,0,0]
#    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c2_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "mimic"
    num_steps = 500
    data = cleo.execute(file_name="mimic",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:74: RuntimeWarning: divide by zero encountered in log
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)
/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:74: RuntimeWarning: invalid value encountered in scalar multiply
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)


90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_

/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:74: RuntimeWarning: invalid value encountered in log
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)


90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_2-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_2-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_2-order_rule_name_mimic
90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_3-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_3-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_3-order_

In [44]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    initial_pops = [0,0,0,0,1,0,0,0]
#    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c4_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "mimic"
    num_steps = 500
    data = cleo.execute(file_name="mimic",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_

In [45]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    initial_pops = [0,0,0,0,1,0,0,0]
#    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c5_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "mimic"
    num_steps = 500
    data = cleo.execute(file_name="mimic",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_

In [46]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    initial_pops = [0,0,0,0,1,0,0,0]
#    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c6_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "mimic"
    num_steps = 500
    data = cleo.execute(file_name="mimic",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_

In [47]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    initial_pops = [0,0,0,0,1,0,0,0]
#    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "cN_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "mimic"
    num_steps = 500
    data = cleo.execute(file_name="mimic",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_

In [48]:
import sys as SYS; SYS.path.insert(0, '../..')
from Scripts import hdf5merge
hdf5merge.merge_hdf5_files("../data/mimic")

In [4]:
Trials = 10
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    #initial_pops = [0,0,0,0,1,0,0,0]
    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c2_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "landscape_maximizes"
    num_steps = 20
    data = cleo.execute(file_name="landscape_maximizes",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:56: RuntimeWarning: invalid value encountered in log
  return 1 / (np.log((1 - pop) / pop))


90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c2_2local_connectivity-unitary_energy_subs

In [6]:
Trials = 10
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    initial_pops = [0,0,0,0,1,0,0,0]
#    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c4_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "landscape_maximizes"
    num_steps = 20
    data = cleo.execute(file_name="landscape_maximizes",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:74: RuntimeWarning: divide by zero encountered in log
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)
/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:74: RuntimeWarning: invalid value encountered in scalar multiply
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)
/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:74: RuntimeWarning: invalid value encountered in log
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)
/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:75: RuntimeWarning: divide by zero encountered in log
  tr_2 = (1 - pop_1) * np.log(1 - pop_2) + (pop_1) * np.log(pop_2)
/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:60: RuntimeWarning: overflow encountered in exp
  pop = 1 / (1 + np.exp(1 / T))


90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c4_2local_connectivity-unitary_energy_subs

In [4]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    initial_pops = [0,0,0,0,1,0,0,0]
#    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c5_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "landscape_maximizes"
    num_steps = 500
    data = cleo.execute(file_name="landscape_maximizes",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:56: RuntimeWarning: invalid value encountered in log
  return 1 / (np.log((1 - pop) / pop))
/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:74: RuntimeWarning: divide by zero encountered in log
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)
/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:74: RuntimeWarning: invalid value encountered in scalar multiply
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)
/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:74: RuntimeWarning: invalid value encountered in log
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)
/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:75: RuntimeWarning: divide by zero encountered in log
  tr_2 = (1 - pop_1) * np.log(1 - pop_2) + (pop_1) * np.log(pop_2)
/Users/uja5020/Documents/GitHub/Qnibble/Notebooks

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subs

/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:75: RuntimeWarning: invalid value encountered in scalar multiply
  tr_2 = (1 - pop_1) * np.log(1 - pop_2) + (pop_1) * np.log(pop_2)


90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_6-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_6-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_6-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_7-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subs

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_19-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_19-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_19-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_20-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_

...

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_45-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_45-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_45-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_46-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_58-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_58-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_58-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_59-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_71-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_71-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_71-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_72-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_84-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_84-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_84-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_85-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_97-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_97-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_97-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_98-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_

In [5]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    initial_pops = [0,0,0,0,1,0,0,0]
#    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c6_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "landscape_maximizes"
    num_steps = 500
    data = cleo.execute(file_name="landscape_maximizes",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subs

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_13-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_13-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_13-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_14-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_26-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_26-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_26-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_27-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_39-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_39-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_39-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_40-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_52-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_52-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_52-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_53-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_65-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_65-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_65-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_66-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_78-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_78-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_78-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_79-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_91-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_91-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_91-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_92-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_

In [8]:
Trials = 10
for unitary_seed in range(Trials):
    #initial_pops = [0.043407, 0.12, 0.09, 0.15, 0.3,0.14, 0.23, 0.026593]



    #initial_pops = [0,0,0,0,1,0,0,0]
    initial_pops = [0.1, 0.1, .1, .1,0.4,0.1,0.1,0.1]

    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "cN_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "landscape_maximizes"
    num_steps = 20
    data = cleo.execute(file_name="landscape_maximizes",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_1-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-cN_2local_connectivity-unitary_energy_subs

In [5]:
import sys as SYS; SYS.path.insert(0, '../..')
from Scripts import hdf5merge
hdf5merge.merge_hdf5_files("../data/landscape_maximizes")

y
